# Filtering in the Frequency Domain
In this notebook, we will go through the basics of filtering in the frequency domain using the Fourier transform.

## Section 0. Preparing the Notebook
We start by importing the necessary libraries and then loading a sample image to work on.

## Section 1. Fourier Transform in NumPy

## Section 2. Filtering in the Frequency Domain

## Section 3. Common Filters

**Note:** There are some methods of filtering in the frequency domain which are mainly aimed at removing noise patterns. You can read about these methods and try them in the "Noise Removal" notebook.

# Scratchbook
You can use this section to try out different codes, without making a mess of the notebook. :)